    - Ollama -> See : https://ollama.com/download
  - For linux users :
    
    For this tutorial, run these instructions
    - Install Ollama : `curl -fsSL https://ollama.com/install.sh | sh`
    - Pull required models : 
        - `ollama pull mayflowergmbh/occiglot-7b-fr-en-instruct` #french llm model
        - `ollama pull sammcj/sfr-embedding-mistral:Q4_K_M` # decent embedding for this use case
    
    -------------------------------------------------------------------------------
  
    Additional informations about Ollama
    - To remove Ollama : https://github.com/ollama/ollama/blob/main/docs/linux.md
    - To stop ollama server : `systemctl stop ollama`
    - To restart server : `systemctl start ollama`
    
    -------------------------------------------------------------------------------
    Due to issues from Ollama's latest versions in weights update, we might want to install older versions of Ollama.
    To install **v.0.1.31** on Linux:
      - `curl -fsSL https://ollama.com/install.sh | sed 's#https://ollama.com/download/ollama-linux-${ARCH}${VER_PARAM}#https://github.com/ollama/ollama/releases/download/v0.1.31/ollama-linux-amd64#' | sh`
            
- Next steps
    - Add source for response provided by the chatbot (e.g., source: from 'Le Challenger' via 'Malijet'. To know more, here are some useful links: links...)
    - Improve model response (accuracy and precision)


In [1]:
from pathlib import Path
import os
import uuid
import pandas as pd
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.vectorstores.chroma import Chroma
from langchain_groq import ChatGroq
# from langchain_community.vectorstores import DocArrayInMemorySearch
# from langchain_openai.embeddings import OpenAIEmbeddings

In [3]:
%env OPENAI_API_KEY=sk-xxx
%env GROQ_API_KEY=gsk_xxx

env: OPENAI_API_KEY=sk-xxx
env: GROQ_API_KEY=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


In [4]:
chat = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
    # api_key="" # Optional if not set as an environment variable
)

system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke({"text": "Explain the importance of low latency for LLMs."})

AIMessage(content="Low latency is crucial for Large Language Models (LLMs) because it directly impacts the user experience, model performance, and overall efficiency of language-based applications. Here are some reasons why low latency is essential for LLMs:\n\n1. **Real-time Interaction**: LLMs are often used in applications that require real-time interaction, such as chatbots, virtual assistants, and language translation systems. Low latency ensures that the model responds quickly to user input, providing a seamless and engaging experience.\n2. **Conversational Flow**: In conversational AI, latency can disrupt the natural flow of conversation. High latency can lead to awkward pauses, making the interaction feel unnatural and frustrating. Low latency helps maintain a smooth conversation, allowing users to engage more naturally with the model.\n3. **Model Performance**: LLMs rely on complex algorithms and massive datasets to generate responses. High latency can lead to increased comput

In [27]:
# ARTICLE_SOURCE_FILE_PATH = Path().resolve().parent /"data" / "malijet" / "source.csv"
CHROMA_DB_PERSIST_PATH = Path().resolve().parent / "data" / "chroma_db_1024"
MODEL_NAME = "mayflowergmbh/occiglot-7b-fr-en-instruct"
# MODEL_NAME = "falcon2:11b-q4_0"

In [4]:
# get nb cpu
os.cpu_count()

16

In [5]:
system_role = "Tu es un expert sur les actualités du Mali et tu parles uniquement français (spécialisé en langue française)."
llm = Ollama(
    model=MODEL_NAME, 
    system=system_role, 
    # num_thread=os.cpu_count()-6
)
llm

Ollama(model='mayflowergmbh/occiglot-7b-fr-en-instruct', system='Tu es un expert sur les actualités du Mali et tu parles uniquement français (spécialisé en langue française).')

## Testing Simple LLM discussion

In [12]:
llm.invoke("Cite moi les noms des présidents du Mali.")

'Les noms des présidents du Mali sont Amadou Toumani Touré, Dioncounda Traoré, Ibrahim Boubacar Keïta et Assimi Goita.\n'

In [13]:
llm.invoke("Quelle est la plus grande crise que la Mali a connue ?")

"La crise la plus grave que le Mali ait connue est sans doute la guerre civile de 2012. Cette conflit a opposé les forces gouvernementales aux rebelles touaregs et à l'Armée malienne pour la libération de l'Azawad (AMAL). La guerre civile a fait des milliers de morts et a provoqué une crise humanitaire majeure dans le pays.\n"

In [14]:
llm.invoke("Who is the most popular scientist in the world?")

'Je ne suis pas capable de répondre à cette question. Je peux vous donner des informations sur les scientifiques célèbres, mais je ne peux pas dire qui est le plus populaire dans le monde entier.'

## 2. Build RAG with CSV file

In [6]:
# loader = CSVLoader(
#     file_path=ARTICLE_SOURCE_FILE_PATH,
#     csv_args={
#         "delimiter": "\t"
#     }
# )

loader = DirectoryLoader("../data/", glob="**/*.csv", loader_cls=CSVLoader)
# load documents
data = loader.load()
data[:3] # three first documents

[Document(metadata={'source': '/home/bouba/Workspace/kounafoni-app/data/malijet/source.csv', 'row': 0}, page_content='title: Fin de la transition militaire: des propositions farfelues du Panel des démocrates\nsource_paper: Info Matin\ndate: 2024-05-16\nlink: https://malijet.com/a_la_une_du_mali/290855-fin-de-la-transition-militaire-des-propositions-farfelues-du-pan.html\ncontent: Des supposés ‘’démocrates’’ maliens qui n’ont pas osé s’afficher dans leur communiqué de presse d’appel à la mobilisation pour la mise en place d’une transition civile auront-ils le courage de fouler le territoire national pour la concrétisation de leur projet ? La déclaration de ce ‘’Panel des démocrates’’ nous rappelle l’affaire de l’opposant malien, Ainea Ibrahim CAMARA, qui s’était proclamé président du Mali depuis la Côte d’Ivoire. Dans son document, le fameux ‘’Panel des démocrates’’ expose des propositions farfelues en demandant que le pays soit dirigé par un chef d’État honorifique dés

In [7]:
pd.Series([i.metadata['source'] for i in data]).value_counts()
# 53 articles extracted
len(data), #pd.read_csv(ARTICLE_SOURCE_FILE_PATH, sep="\t").shape[0]

(57, 57)

## Let's see if the splitter is necessary

### Without splitter

In [8]:
def get_length_info(list_of_documents, splitters=None):
    
    if splitters is None:
        splitters = [' ', '.']
    
    ## search the content length statistics (nb characters)
    print('-'*10, "For character length", '-'*10)
    display(pd.Series([len(document.page_content) for document in list_of_documents]).describe())
    
    ## search the words length statistics (nb words)
    print('-'*10, "Length of words (nb characters) in the corpus", '-'*10)
    _res = list()
    for document in list_of_documents:
        _res += pd.Series(document.page_content.split(splitters[0])).apply(len).tolist()
    display(pd.Series(_res).describe())
    
    ## search the sentence length statistics (nb words)
    print('-'*10, "How many words in each doc", '-'*10)
    display(pd.Series([len(doc.page_content.split(splitters[0])) for doc in list_of_documents]).describe())
    
    ## search the sentence length statistics (nb words)
    print('-'*10, "For how many sentences in in each doc", '-'*10)
    display(pd.Series([len(doc.page_content.split(splitters[1])) for doc in list_of_documents]).describe())
    
    ## search the sentence length statistics (nb words)
    print('-'*10, "For sentences length in each doc", '-'*10)
    display(pd.Series([len(sentence) for document in list_of_documents for sentence in document.page_content.split('.')]).describe())
    
    return _res

In [9]:
res = get_length_info(data)

---------- For character length ----------


count       57.000000
mean      4890.017544
std       5142.846080
min        373.000000
25%       2038.000000
50%       3523.000000
75%       6219.000000
max      30532.000000
dtype: float64

---------- Length of words (nb characters) in the corpus ----------


count    42220.000000
mean         5.603221
std          5.570993
min          0.000000
25%          2.000000
50%          4.000000
75%          8.000000
max        123.000000
dtype: float64

---------- How many words in each doc ----------


count      57.000000
mean      740.701754
std       804.136204
min        30.000000
25%       304.000000
50%       559.000000
75%       929.000000
max      4616.000000
dtype: float64

---------- For how many sentences in in each doc ----------


count     57.000000
mean      30.666667
std       39.169655
min        4.000000
25%       14.000000
50%       21.000000
75%       35.000000
max      287.000000
dtype: float64

---------- For sentences length in each doc ----------


count    1748.000000
mean      158.489703
std       148.669395
min         0.000000
25%        76.000000
50%       132.500000
75%       207.250000
max      2386.000000
dtype: float64

## With splitter

In [10]:
pd.Series(res).max(), pd.Series(res).quantile(.99)

(123, 16.0)

In [11]:
quantile = 387 # first version
maxi_character_per_sentence = pd.Series([len(sentence) for document in data for sentence in document.page_content.split('.')]).max().astype(int)
maxi_character_per_words = 20 # over 16 to make sure to get all-content overlap
quantile, maxi_character_per_sentence, maxi_character_per_words

(387, 2386, 20)

How do I determine how many characters form a meaningful chunk (i.e., understandable and self-sufficient information)?
We will assume that each sentence could be a meaningful chunk. The idea is to have a count distribution of sentences length in our corpus to make a quick decision.
- We might choose 400 (similar to the quantile q95) as chunk size to be more flexible. In this case, we stay with 387 as a chunk size
- Also, in this case, we'll set 20 as the maximum length of character possible in a sentence

In [12]:
## test the document splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=quantile, # selected quantile
    chunk_overlap=maxi_character_per_words,
    separators=["\n\n", "\n", ". ", " ", ""], # specify that sentence split (by dot) is more important than space & other
    keep_separator=False # drop the separators from the document after split
)
documents = text_splitter.split_documents(documents=data)
len(documents)

998

In [13]:
documents[8:20]

[Document(metadata={'source': '/home/bouba/Workspace/kounafoni-app/data/malijet/source.csv', 'row': 0}, page_content='Sans ambages, ce panel des démocrates appelle à un soulèvement contre la transition en cours en vue de rétablir « la souveraineté populaire et l’ordre constitutionnel, à soutenir l’investiture du gouvernement de transition en exil et à participer massivement à la manifestation populaire des démocrates maliens le 28 juin 2024 à Bamako, dans les autres localités du Mali et à'),
 Document(metadata={'source': '/home/bouba/Workspace/kounafoni-app/data/malijet/source.csv', 'row': 0}, page_content='du Mali et à l’étranger..'),
 Document(metadata={'source': '/home/bouba/Workspace/kounafoni-app/data/malijet/source.csv', 'row': 0}, page_content='». Dernière ce projet machiavélique qui n’est pas visiblement soutenu par certains hommes politiques joints par nos soins, malgré leur divergence avec les autorités de la transition, se cache le désir de ces ‘’démocrat

In [14]:
documents[7]
res = get_length_info(documents)

---------- For character length ----------


count    998.000000
mean     279.109218
std       92.515471
min       14.000000
25%      228.250000
50%      299.000000
75%      355.000000
max      387.000000
dtype: float64

---------- Length of words (nb characters) in the corpus ----------


count    42253.000000
mean         5.616075
std          5.316811
min          0.000000
25%          2.000000
50%          4.000000
75%          8.000000
max        118.000000
dtype: float64

---------- How many words in each doc ----------


count    998.000000
mean      42.337675
std       15.731167
min        1.000000
25%       32.000000
50%       46.000000
75%       55.000000
max       79.000000
dtype: float64

---------- For how many sentences in in each doc ----------


count    998.000000
mean       1.923848
std        1.322672
min        1.000000
25%        1.000000
50%        2.000000
75%        2.000000
max       13.000000
dtype: float64

---------- For sentences length in each doc ----------


count    1920.000000
mean      144.598437
std       105.372016
min         0.000000
25%        68.000000
50%       128.000000
75%       205.250000
max       387.000000
dtype: float64

 We move from 4616 words maxi per doc to 79 words per doc. That might be relevant for the final framework of RAG.

In [15]:
[doc for doc in documents[:5]]

['title: Fin de la transition militaire: des propositions farfelues du Panel des démocrates\nsource_paper: Info Matin\ndate: 2024-05-16\nlink: https://malijet.com/a_la_une_du_mali/290855-fin-de-la-transition-militaire-des-propositions-farfelues-du-pan.html',
 'content: Des supposés ‘’démocrates’’ maliens qui n’ont pas osé s’afficher dans leur communiqué de presse d’appel à la mobilisation pour la mise en place d’une transition civile auront-ils le courage de fouler le territoire national pour la concrétisation de leur projet ? La déclaration de ce ‘’Panel des démocrates’’ nous rappelle l’affaire de l’opposant malien, Ainea Ibrahim',
 'Ainea Ibrahim CAMARA, qui s’était proclamé président du Mali depuis la Côte d’Ivoire',
 'Dans son document, le fameux ‘’Panel des démocrates’’ expose des propositions farfelues en demandant que le pays soit dirigé par un chef d’État honorifique désigné chaque année par exercice tournant parmi les sénateurs...Autre hic, alors que certai

In [19]:
[doc.page_content for doc in documents[37:40]]

## Embedding and Vector store

In [22]:
# embeddings_llm = OllamaEmbeddings(model=MODEL_NAME) #mistral or Occiglot
# embeddings_llm = OllamaEmbeddings(model="mayflowergmbh/occiglot-7b-fr-en-instruct")
# embeddings_llm = OllamaEmbeddings(model="snowflake-arctic-embed")
# embeddings_llm = OpenAIEmbeddings()

# embeddings_llm = OllamaEmbeddings(model="bge-large:335m-en-v1.5-fp16")

embeddings_llm = OllamaEmbeddings(model="bge-m3:567m-fp16")
# Set few params if needed
embeddings_llm.show_progress = True
embeddings_llm.num_thread = os.cpu_count() - 8 # 8 in my case
# embeddings_llm.top_k = 10
# embeddings_llm.top_p = .5

embeddings_llm

OllamaEmbeddings(base_url='http://localhost:11434', model='bge-m3:567m-fp16', embed_instruction='passage: ', query_instruction='query: ', mirostat=None, mirostat_eta=None, mirostat_tau=None, num_ctx=None, num_gpu=None, num_thread=8, repeat_last_n=None, repeat_penalty=None, temperature=None, stop=None, tfs_z=None, top_k=None, top_p=None, show_progress=True, headers=None, model_kwargs=None)

In [23]:
# Exemple
documents[3], len(documents[3].page_content)

(Document(metadata={'source': '/home/bouba/Workspace/kounafoni-app/data/malijet/source.csv', 'row': 0}, page_content='Dans son document, le fameux ‘’Panel des démocrates’’ expose des propositions farfelues en demandant que le pays soit dirigé par un chef d’État honorifique désigné chaque année par exercice tournant parmi les sénateurs...Autre hic, alors que certains démocrates exigent le retour immédiat à l’ordre constitutionnel, eux ils proposent encore une autre transition dite civile de'),
 386)

In [24]:
len(embeddings_llm.embed_query(documents[3].page_content))


OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.72s/it]


1024

In [25]:
# test to embed a query
embeddings_llm.embed_query(documents[3].page_content)[:7], len(embeddings_llm.embed_query(documents[3].page_content))


OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.20it/s]

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  1.47it/s]


([-0.6823922991752625,
  0.23682698607444763,
  -0.46551015973091125,
  -0.07770388573408127,
  -0.7255619764328003,
  -1.188683032989502,
  1.1188299655914307],
 1024)

## Selection of Vector Store

Thanks to this brand-new article from Google, we can safely choose any open source Vector Store, make it available to Google NFS Filestore and access it easily through mounting filestore in Cloud Run (see section 3): https://cloud.google.com/blog/products/serverless/introducing-cloud-run-volume-mounts?hl=en

In [28]:
# The embedding is a very large dimension of 4096, so this will take a real long time
# Database creation
# now 1024 embedding to test Cloud Run
# db = Chroma.from_documents(
#     documents=documents[:2],
#     embedding=embeddings_llm,
#     persist_directory=CHROMA_DB_PERSIST_PATH.as_posix(),
# )
# db


OllamaEmbeddings: 100%|██████████| 2/2 [00:01<00:00,  1.36it/s]


In [29]:
db = Chroma(
    persist_directory=CHROMA_DB_PERSIST_PATH.as_posix(),
    embedding_function=embeddings_llm,
)

persisted_ids = db.get()["ids"]

new_documents_to_embed_df = pd.DataFrame({
    "single_id": [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in documents],
    "document": documents
})

# to keep only different documents (i.e. chunks)
new_documents_to_embed_df.drop_duplicates(subset="single_id", inplace=True)

# Keep only documents not already embedded
new_documents_to_embed_df = new_documents_to_embed_df.query(f"single_id not in {persisted_ids}")

if new_documents_to_embed_df.empty:
    print("No documents to embed")
else:
    print("Embedding documents...")
    display(new_documents_to_embed_df.head())
    db.add_documents(
        documents=new_documents_to_embed_df.document.tolist(), 
        embedding=embeddings_llm, 
        ids=new_documents_to_embed_df.single_id.tolist(), 
        persist_directory=CHROMA_DB_PERSIST_PATH.as_posix(),
    )


Embedding documents...


/home/bouba/Workspace/kounafoni-app/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


,single_id,document
0,462cc7cc-1d47-5cbd-a6f4-5d1138eb6701,page_content='title: Fin de la transition mili...
1,59a6057c-68fa-5fc3-b369-48b2824bcbbd,page_content='content: Des supposés ‘’démocr...
2,2733bc82-21dd-55c8-a5c0-e83cdea28fef,"page_content='Ainea Ibrahim CAMARA, qui s’éta..."
3,a48b629b-14c1-5633-835e-b093eef039a5,"page_content='Dans son document, le fameux ‘’P..."
4,f98844d1-dd20-583c-b255-ebd919d1b112,page_content='dite civile de 5 ans' metadata={...



OllamaEmbeddings:   0%|          | 0/1 [29:56<?, ?it/s]


In [30]:
db_check = Chroma(
    persist_directory=CHROMA_DB_PERSIST_PATH.as_posix(),
    embedding_function=embeddings_llm,
)
db_check.get()

{'ids': ['005140f5-1ba0-5ebb-b0a3-47b12167928b',
  '007e1a57-1526-51b0-98a7-709fdcad4050',
  '008ebdd9-ad2c-50f4-b2e2-7487629a7b41',
  '0092e22d-f5fe-5103-8dd4-1b1ee73f2465',
  '00c95598-8563-5226-a1bf-42efe8936de1',
  '010669f1-6ce8-5b61-b576-c1680c66ff8e',
  '01382f66-8a37-5e81-a4f0-cfdf0216b708',
  '016a44d5-97a6-576c-afaa-16b0fc774a8d',
  '016fd3eb-bcae-5f19-ac92-e414515bac43',
  '01a87216-0f73-5431-9eb0-329a5b514499',
  '01d3ed7b-a613-5a42-9c3f-c645fbc652ee',
  '01fbe6ae-acbd-5465-8400-816704ca4bbd',
  '01fec80d-7dc4-5b91-9eb5-0039adc9a56d',
  '03424773-cff0-5369-a90c-5b38f40be9bd',
  '037250a5-29f0-56fb-a455-73abcaab7abc',
  '03c83b04-7ec6-5f9d-acf7-b432f36e1768',
  '03f053f6-23ee-5753-b9b4-36af3e3f2cf8',
  '040e4bf4-53ff-5934-8cd6-c51c6ee9f0a9',
  '04cd12c6-6626-58f6-ba97-2b3d4af971d0',
  '05d60865-7f47-548f-bc18-138f0922fb7c',
  '06a49462-e3c4-5627-8aed-469c2f27dd29',
  '06fa03cb-dce6-5648-9bcd-413d95cc7b43',
  '07323be0-c018-535c-bd7f-4ecf53325110',
  '07bba7a1-8976-5cfc-9362-

In [42]:
# db_check.get(include=['embeddings'])

In [62]:
# db_check.delete_collection()

In [23]:
# In memory vector store test!

# Took more than 36min for 46 documents only; wow, I interrupted with the keyboard!
# For model name as occiglot or mistral or sfr-embedding first on retrieval
# db = DocArrayInMemorySearch.from_documents(documents, embedding=embeddings_llm)

In [26]:
# for snowflake
# db2 = DocArrayInMemorySearch.from_documents(documents, embedding=embeddings_llm) 

In [27]:
# for openai
# db4 = DocArrayInMemorySearch.from_documents(documents, embedding=embeddings_llm)

In [28]:
# load db from disk
# db3 = Chroma(persist_directory=CHROMA_DB_PERSIST_PATH.as_posix(), embedding_function=embeddings_llm)
# db3

## Retriever for RAG

In [29]:
retriever = db.as_retriever(search_kwargs={"k": 7}) # Occiglot or sfr-embed
# retriever = db2.as_retriever(search_kwargs={"k": 10}) # snowflake
# retriever = db4.as_retriever(search_kwargs={"k": 10}) #openAI
# retriever = db3.as_retriever(search_kwargs={"k": 5})
retriever

In [30]:
# Test retriever
query = "AES" 
retriever.invoke(query), len(retriever.invoke(query))

In [31]:
# Prompt template
template = """
Réponds à la question uniquement grâce au contexte suivant et uniquement en langue française. N'hésites pas à détailler ta réponse. A la fin de ta réponse, mets en bas la source de média qui t'as permis d'avoir ces réponses.
Si tu n'as pas de réponse explicite dans le contexte, réponds "Je n'ai pas assez d'informations pour répondre correctement à votre question.".

Contexte : {context}

Question : {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [32]:
def format_docs(docs):
    # print(docs)
    # print('*'*15)
    return "\n\n".join([d.page_content for d in docs])

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    # | llm
    | ChatGroq(temperature=0, model="llama3-70b-8192")
    | StrOutputParser()
)


In [33]:
questions = [
    # "Qui est Oumar Diarra ?",
    # "Quels sont les actions de l'EUTM ?",
    # "Cite-moi les recommandations retenues lors du dialogue inter malien",
    # "Quand finit la mission de l'union européenne ?",
    # "Actualités sur la DIRPA",
    # "Parle-moi de l'agriculture au Mali",
    # "Que font les FAMA actuellement ?"
    
    # "Parle moi du nouveau vérificateur général",
    # "Résume moi en quelques points les dernières actualités maintenant",
    # "Où en est la relation Mali et Russie ?",
    # "Qui est Bassaro Haïdara ?",
    # "Qu'est ce que Assimi a fait récemment ?",
    # "Le dialogue inter malien est il terminé ?",
    # "Qui sont les membres de l'AES ?",
    # "Comment a été la journée du 1er Mai au Mali ?",
    # "Donne moi la date la plus récente des informations dont tu disposes",
    # "Qu'en est il de la crise sécuritaire au Mali ?",
    # "La Belgique a t elle récemment collaborée avec le Mali ?" # question bonus (must return I dont know),
    
    "Y a t il des tensions entre le Mali et le Danemark ?",
    "Quelles les implications de l'Algérie à la guerre au nord du Mali ?",
    "Qu'a fait le président Assimi le 15 Aout ?",
    "Fais moi un bilan des inondations du Mali."
]
len(questions)

In [34]:
for q in questions:
    print(q)
    print(chain.invoke(q))
    print('-'*50, "\n")
# for open source models
for q in questions:
    print(q)
    print(chain.invoke(q))
    print('-'*50, "\n")

## Récapitulatif

- 8 bons retrieval sur 11 questions [V, V, V, X, V, V, X, V, X, V, V] : V → True response and X → Wrong response
- 4 bonnes réponses sur 11 questions [X, V, V, X, V, X, X, X, X, X, V] :  V → True response and X → Wrong response
- Bonne réponse à la question Bonus !
- Questions potentielles à rajouter :
    - Quand est ce que Abdoulaye Diop rencontrera ses homologues ?
    -  Quoi de prévu le 28 juin à Bamako ?
    - Quelle célébrité le président a rencontré ?
    - À quand la transition civile au Mali ?

# Some results from OpenAIEmbeddings, far better result

In [40]:
# Result from OpenAI Embeddings
for q in questions:
    print(q)
    print(chain.invoke(q))
    print('-'*50, "\n")